In [7]:
#1. How many users, activities and trackpoints are there in the dataset (after it is inserted into the database).

from DbConnector import DbConnector
db = DbConnector() 
cursor = db.cursor

cursor.execute("SELECT COUNT(*) FROM User")
num_users = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM Activity")
num_activities = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM TrackPoint")
num_trackpoints = cursor.fetchone()[0]

print(f"Users: {num_users}, Activities: {num_activities}, Trackpoints: {num_trackpoints}")

Users: 182, Activities: 16048, Trackpoints: 9681756


In [10]:

#2. Find the average number of activities per user.

from DbConnector import DbConnector
db = DbConnector() 
cursor = db.cursor

cursor.execute("SELECT AVG(activity_count) FROM (SELECT COUNT(*) as activity_count FROM Activity GROUP BY user_id) as temp")
average_activities = cursor.fetchone()[0]

print(f"Average number of activities per user: {average_activities}")



Average number of activities per user: 92.7630


In [11]:
#3. Find the top 20 users with the highest number of activities. 

cursor.execute("""
    SELECT user_id, COUNT(*) as activity_count
    FROM Activity
    GROUP BY user_id
    ORDER BY activity_count DESC
    LIMIT 20
""")
top_users = cursor.fetchall()

for user in top_users:
    print(f"User ID: {user[0]}, Activity Count: {user[1]}")

User ID: 128, Activity Count: 2102
User ID: 153, Activity Count: 1793
User ID: 25, Activity Count: 715
User ID: 163, Activity Count: 704
User ID: 62, Activity Count: 691
User ID: 144, Activity Count: 563
User ID: 41, Activity Count: 399
User ID: 85, Activity Count: 364
User ID: 4, Activity Count: 346
User ID: 140, Activity Count: 345
User ID: 167, Activity Count: 320
User ID: 68, Activity Count: 280
User ID: 17, Activity Count: 265
User ID: 3, Activity Count: 261
User ID: 14, Activity Count: 236
User ID: 126, Activity Count: 215
User ID: 30, Activity Count: 210
User ID: 112, Activity Count: 208
User ID: 11, Activity Count: 201
User ID: 39, Activity Count: 198


In [12]:
#4. Find all users who have taken a taxi. 

cursor.execute("""
    SELECT DISTINCT user_id
    FROM Activity
    WHERE transportation_mode = 'taxi'
""")
taxi_users = cursor.fetchall()

print("Users who have taken a taxi:")
for user in taxi_users:
    print(user[0])

Users who have taken a taxi:
10
111
114
126
128
139
153
163
167
175
21
52
56
58
62
65
78
80
84
85
98


In [13]:
#5. Find all types of transportation modes and count how many activities that are tagged with these transportation mode labels. Do not count the rows where the mode is null.
cursor.execute("""
    SELECT transportation_mode, COUNT(*)
    FROM Activity
    WHERE transportation_mode IS NOT NULL
    GROUP BY transportation_mode
""")
modes_count = cursor.fetchall()

for mode, count in modes_count:
    print(f"{mode}: {count} activities")

taxi: 125 activities
walk: 1008 activities
bus: 471 activities
bike: 619 activities
car: 493 activities
run: 1 activities
train: 2 activities
subway: 190 activities
airplane: 4 activities
boat: 1 activities


In [15]:
#6. a) Find the year with the most activities. 

cursor.execute("""
    SELECT YEAR(start_date_time) as year, COUNT(*) as activity_count
    FROM Activity
    GROUP BY year
    ORDER BY activity_count DESC
    LIMIT 1
""")
most_activities_year = cursor.fetchone()

print(f"Year with the most activities: {most_activities_year[0]} with {most_activities_year[1]} activities")

Year with the most activities: 2008 with 5895 activities


In [103]:
#6 b) Is this also the year with most recorded hours?
cursor.execute("""
    SELECT YEAR(start_date_time) as year, 
           SUM(TIMESTAMPDIFF(SECOND, start_date_time, end_date_time)) / 3600 AS total_hours
    FROM Activity
    GROUP BY year
    ORDER BY total_hours DESC
    LIMIT 1;
""")
most_hours_year = cursor.fetchone()

print(f"Year with the most recorded hours: {most_hours_year[0]} with {most_hours_year[1]:.2f} hours")


Year with the most recorded hours: 2009 with 11612.42 hours


In [104]:
#7. Find the total distance (in km) walked in 2008, by user with id=112

from DbConnector import DbConnector
db = DbConnector() 
cursor = db.cursor

from haversine import haversine, Unit

cursor.execute("""
    SELECT id
    FROM Activity
    WHERE user_id = 112 AND YEAR(start_date_time) = 2008 AND transportation_mode = 'walk'
""")
activities = cursor.fetchall()

total_distance = 0

for activity in activities:
    activity_id = activity[0]
    
    cursor.execute("""
        SELECT lat, lon
        FROM TrackPoint
        WHERE activity_id = %s
        ORDER BY date_time
    """, (activity_id,))
    trackpoints = cursor.fetchall()
    
    activity_distance = 0
    for i in range(1, len(trackpoints)):
        lat1, lon1 = trackpoints[i - 1]
        lat2, lon2 = trackpoints[i]
        activity_distance += haversine((lat1, lon1), (lat2, lon2), unit=Unit.KILOMETERS)
    
    total_distance += activity_distance

print(f"Total distance walked by user 112 in 2008: {total_distance} km")


Total distance walked by user 112 in 2008: 256.38282457874544 km


In [106]:
#8. Find the top 20 users who have gained the most altitude meters.
from DbConnector import DbConnector
db = DbConnector() 
cursor = db.cursor
import pandas as pd
import tabulate


cursor.execute("""
    SELECT user_id, SUM(altitude_gain) as total_gain
    FROM (
        SELECT user_id, GREATEST(0, altitude - LAG(altitude) OVER (PARTITION BY activity_id ORDER BY date_time)) as altitude_gain
        FROM TrackPoint
        JOIN Activity ON TrackPoint.activity_id = Activity.id
        WHERE altitude >0
    ) as temp
    GROUP BY user_id
    ORDER BY total_gain DESC
    LIMIT 20
""")
altitude_users = cursor.fetchall()


data = {
    'User ID': [user[0] for user in altitude_users],
    'Total Altitude Gain (meters)': [float(user[1]) * 0.3048 for user in altitude_users]  # Convert feet to meters
}

df = pd.DataFrame(data)

print(df)


   User ID  Total Altitude Gain (meters)
0      128                   555588.2208
1      153                   491810.0400
2        4                   294319.4520
3        3                   203890.4736
4       85                   185603.3880
5      163                   185340.6504
6       30                   148108.1112
7      144                   143911.9296
8       39                   124493.1216
9       62                   118092.9312
10      41                   113247.5256
11      84                   113236.5528
12     167                   104664.9672
13       0                   103207.4136
14       2                   101600.5080
15      25                    82172.5560
16     126                    74829.6192
17     140                    68096.5872
18      17                    54443.9856
19      22                    52231.1376


In [5]:
#9. Find all users who have invalid activities and the number of invalid activities per user. 

from DbConnector import DbConnector
db = DbConnector() 
cursor = db.cursor

users_of_invalid_activies = []

cursor.execute("""
    SELECT Activity.user_id, COUNT(DISTINCT temp.activity_id) as invalid_activities
    FROM (
        SELECT TrackPoint.activity_id, 
               TIMESTAMPDIFF(SECOND, 
                             LAG(TrackPoint.date_time) OVER (PARTITION BY TrackPoint.activity_id ORDER BY TrackPoint.date_time), 
                             TrackPoint.date_time) as time_diff
        FROM TrackPoint
    ) as temp
    JOIN Activity ON Activity.id = temp.activity_id
    WHERE temp.time_diff >= 300
    GROUP BY Activity.user_id
    ORDER BY invalid_activities DESC
""")


all_invalid_activities = cursor.fetchall()


for user in all_invalid_activities:
    users_of_invalid_activies.append(user[0])

# Displaying only the top 20 users with the highest number of invalid activities
print("Top 20 Users with invalid activities and the number of invalid activities:")
for user in all_invalid_activities[:20]:  # Slicing to get the top 20
    print(f"User ID: {user[0]}, Invalid Activities: {user[1]}")

#print(f"Total number of users with invalid activities: {(users_of_invalid_activies)} ")
print(f" Numbers of users with invalid activities are {len(users_of_invalid_activies)}")



Top 20 Users with invalid activities and the number of invalid activities:
User ID: 128, Invalid Activities: 720
User ID: 153, Invalid Activities: 557
User ID: 25, Invalid Activities: 263
User ID: 62, Invalid Activities: 249
User ID: 163, Invalid Activities: 233
User ID: 4, Invalid Activities: 219
User ID: 41, Invalid Activities: 201
User ID: 85, Invalid Activities: 184
User ID: 3, Invalid Activities: 179
User ID: 144, Invalid Activities: 157
User ID: 39, Invalid Activities: 147
User ID: 68, Invalid Activities: 139
User ID: 167, Invalid Activities: 134
User ID: 17, Invalid Activities: 129
User ID: 14, Invalid Activities: 118
User ID: 30, Invalid Activities: 112
User ID: 126, Invalid Activities: 105
User ID: 0, Invalid Activities: 101
User ID: 92, Invalid Activities: 101
User ID: 37, Invalid Activities: 100
 Numbers of users with invalid activities are 171


In [91]:
#10.Find the users who have tracked an activity in the Forbidden City of Beijing.

from DbConnector import DbConnector
db = DbConnector() 
cursor = db.cursor

cursor.execute("""
    SELECT DISTINCT user_id
    FROM TrackPoint
    JOIN Activity ON TrackPoint.activity_id = Activity.id
    WHERE ABS(lat - 39.916) < 0.001 AND ABS(lon - 116.397) < 0.001
""")
forbidden_city_users = cursor.fetchall()

print("Users who have tracked an activity in the Forbidden City:")
for user in forbidden_city_users:
    print(f"User ID: {user[0]}")


Users who have tracked an activity in the Forbidden City:
User ID: 131
User ID: 18
User ID: 19
User ID: 4


In [109]:

#11.Find all users who have registered transportation_mode and their most used transportation_mode. 

cursor.execute("""
    SELECT user_id, transportation_mode
    FROM (
        SELECT user_id, transportation_mode, 
               ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY COUNT(*) DESC, transportation_mode ASC) as rn
        FROM Activity
        WHERE transportation_mode IS NOT NULL
        GROUP BY user_id, transportation_mode
    ) as temp
    WHERE rn = 1
    ORDER BY user_id
""")
most_used_transport_mode = cursor.fetchall()

print("Users and their most used transportation mode:")
for user in most_used_transport_mode:
    print(f"User ID: {user[0]}, Most Used Mode: {user[1]}")


Users and their most used transportation mode:
User ID: 10, Most Used Mode: taxi
User ID: 101, Most Used Mode: car
User ID: 102, Most Used Mode: walk
User ID: 105, Most Used Mode: walk
User ID: 107, Most Used Mode: walk
User ID: 108, Most Used Mode: walk
User ID: 111, Most Used Mode: taxi
User ID: 112, Most Used Mode: walk
User ID: 114, Most Used Mode: taxi
User ID: 115, Most Used Mode: car
User ID: 117, Most Used Mode: walk
User ID: 125, Most Used Mode: bike
User ID: 126, Most Used Mode: walk
User ID: 128, Most Used Mode: car
User ID: 129, Most Used Mode: bike
User ID: 136, Most Used Mode: walk
User ID: 138, Most Used Mode: bike
User ID: 139, Most Used Mode: bike
User ID: 141, Most Used Mode: walk
User ID: 144, Most Used Mode: walk
User ID: 153, Most Used Mode: walk
User ID: 161, Most Used Mode: walk
User ID: 163, Most Used Mode: bike
User ID: 167, Most Used Mode: bike
User ID: 170, Most Used Mode: walk
User ID: 175, Most Used Mode: bus
User ID: 179, Most Used Mode: walk
User ID: 20, 